In [7]:
import numpy as np
from netCDF4 import Dataset
import h5py
import time
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import multiprocessing as mp
import time
from obspy.core import Stream, Trace, UTCDateTime, Stats
import xarray as xr
import os

In [30]:
model_id = 29999
run_path = f"./{model_id:0>4d}"
print(run_path)

./29999


In [38]:
np.linspace(0,2.98,150)

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
       0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
       0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
       0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  , 1.02, 1.04, 1.06, 1.08,
       1.1 , 1.12, 1.14, 1.16, 1.18, 1.2 , 1.22, 1.24, 1.26, 1.28, 1.3 ,
       1.32, 1.34, 1.36, 1.38, 1.4 , 1.42, 1.44, 1.46, 1.48, 1.5 , 1.52,
       1.54, 1.56, 1.58, 1.6 , 1.62, 1.64, 1.66, 1.68, 1.7 , 1.72, 1.74,
       1.76, 1.78, 1.8 , 1.82, 1.84, 1.86, 1.88, 1.9 , 1.92, 1.94, 1.96,
       1.98, 2.  , 2.02, 2.04, 2.06, 2.08, 2.1 , 2.12, 2.14, 2.16, 2.18,
       2.2 , 2.22, 2.24, 2.26, 2.28, 2.3 , 2.32, 2.34, 2.36, 2.38, 2.4 ,
       2.42, 2.44, 2.46, 2.48, 2.5 , 2.52, 2.54, 2.56, 2.58, 2.6 , 2.62,
       2.64, 2.66, 2.68, 2.7 , 2.72, 2.74, 2.76, 2.78, 2.8 , 2.82, 2.84,
       2.86, 2.88, 2.9 , 2.92, 2.94, 2.96, 2.98])

In [36]:
# def read_seismogram(Run_dir):
Run_dir = "../Runs/LatinSphericalHarmonicsElasticBall0009"
BallRadius = 1000
nlat, nlon = 37, 37
StartTime, EndTime = 0.0, 3.0   # note in Second
resample_rate = 0.02    # note in Second
lowpass_freq = 20.    # note in Hz
PointPerTrace = int((EndTime - StartTime)/resample_rate)

# initial wave & location array
wave_disp = np.ndarray((nlat, nlon, 3, PointPerTrace)) #nlat, nlon, ncomponent, PointPerTrace
station_coords_cartesian = np.ndarray((nlat, nlon, 3))
station_coords_spherical = np.ndarray((nlat, nlon, 3))

StationInfo = np.loadtxt(f"{Run_dir}/input/Synthetic_Stations_Ball.txt",dtype=str, skiprows=3)
stalatlon_dict = {}
# make station coordinate dict
for item in StationInfo:
    stkey = item[1]+'.'+item[0]
    x = float(item[2])
    y = float(item[3])
    depth = float(item[5])
    if stkey not in stalatlon_dict.keys():
        stalatlon_dict[stkey] = []
    stalatlon_dict[stkey].append((x, y, depth))

# read rank-station info
rank_station_info = np.loadtxt(f"{Run_dir}/output/stations/Synthetic_Stations/rank_station.info", dtype=str, skiprows=1)
# dict: mpi-rank -> [station keys]
rank_station_dict = {}
for item in rank_station_info:
    rank = item[0]
    stkey = item[1]
    inrank_index = item[2]
    # initialize with an empty array if rank does not exists in rank_station_dict
    if rank not in rank_station_dict.keys():
        rank_station_dict[rank] = []
    # append the station
    rank_station_dict[rank].append([stkey, inrank_index])

# loop over mpi-ranks to read data
for rank in rank_station_dict.keys():
    f = Dataset(f"{Run_dir}/output/stations/Synthetic_Stations/axisem3d_synthetics.nc.rank%s" %rank, 'r')
    time = np.array(f.variables['data_time'][:])

    for [StationName, inrank_index] in rank_station_dict[rank]:
        lat, lon = stalatlon_dict[StationName][0][0], stalatlon_dict[StationName][0][1]
        ilat, ilon = int((lat+90)/5), int((lon+180)/10)
        colat = 90 - lat
        theta = np.radians(colat)
        phi = np.radians(lon)
        stadepth = stalatlon_dict[StationName][0][2]
        stax = (BallRadius - stadepth)*np.sin(theta)*np.cos(phi)
        stay = (BallRadius - stadepth)*np.sin(theta)*np.sin(phi)
        staz = (BallRadius - stadepth)*np.cos(theta)
        station_coords_cartesian[ilat, ilon,:] = [stax,stay,staz]
        station_coords_spherical[ilat, ilon,:] = [BallRadius - stadepth,theta,phi]
        # trace header
        stats = Stats()
        stats.starttime = UTCDateTime(time[0])
        stats.delta = UTCDateTime(time[1] - time[0])
        stats.npts = len(time)
        # stream
        stream = Stream()
        for ich, ch in enumerate('RTZ'):
            stats.channel = ch  
            # default unit is km
            stream.append(Trace(f.variables['data_wave'][int(inrank_index)][ich], header=stats))
        stream.filter('lowpass', freq=lowpass_freq)
        stream.resample(1/resample_rate)
        # stream = stream.slice(UTCDateTime(int(arrivals[0].time)+StartTime), UTCDateTime(int(arrivals[0].time)+EndTime))
        stream = stream.slice(UTCDateTime(StartTime), UTCDateTime(EndTime))

        wave_disp[ilat, ilon, 0, :] = stream[0].data[0:PointPerTrace]
        wave_disp[ilat, ilon, 1, :] = stream[1].data[0:PointPerTrace]
        wave_disp[ilat, ilon, 2, :] = stream[2].data[0:PointPerTrace]
        wave_time = np.array(stream[0].times())
        # print(wave_time)

    # wave_disp[ilat, ilon, :, :] = np.array(f.variables['data_wave'][int(inrank_index)][:])
    f.close()

return wave_disp #nlat nlon, ncomponent, npoint

SyntaxError: 'return' outside function (2242709987.py, line 83)

In [33]:
PointPerTrace

149

In [34]:
wave_time

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
       0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
       0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
       0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  , 1.02, 1.04, 1.06, 1.08,
       1.1 , 1.12, 1.14, 1.16, 1.18, 1.2 , 1.22, 1.24, 1.26, 1.28, 1.3 ,
       1.32, 1.34, 1.36, 1.38, 1.4 , 1.42, 1.44, 1.46, 1.48, 1.5 , 1.52,
       1.54, 1.56, 1.58, 1.6 , 1.62, 1.64, 1.66, 1.68, 1.7 , 1.72, 1.74,
       1.76, 1.78, 1.8 , 1.82, 1.84, 1.86, 1.88, 1.9 , 1.92, 1.94, 1.96,
       1.98, 2.  , 2.02, 2.04, 2.06, 2.08, 2.1 , 2.12, 2.14, 2.16, 2.18,
       2.2 , 2.22, 2.24, 2.26, 2.28, 2.3 , 2.32, 2.34, 2.36, 2.38, 2.4 ,
       2.42, 2.44, 2.46, 2.48, 2.5 , 2.52, 2.54, 2.56, 2.58, 2.6 , 2.62,
       2.64, 2.66, 2.68, 2.7 , 2.72, 2.74, 2.76, 2.78, 2.8 , 2.82, 2.84,
       2.86, 2.88, 2.9 , 2.92, 2.94, 2.96, 2.98])

In [35]:
len(wave_time)

150

In [ ]:
wave_disp = read_seismogram('/raid1/TomoNetData/Runs/LatinSphericalHarmonicsPREM0005')

In [2]:
def read_element_output(data_dir, load_wave_data=True):
    ################ open files ################
    # filenames
    nc_fnames = [f for f in os.listdir(data_dir) if 'axisem3d_synthetics.nc' in f]
    # print('files to open: ', nc_fnames)

    # open files
    nc_files = []
    for nc_fname in nc_fnames:
        nc_files.append(xr.open_dataset(data_dir + '/' + nc_fname))
    
    ################ variables that are the same in the datasets ################
    # read Na grid (all azimuthal dimensions)
    na_grid = nc_files[0].data_vars['list_na_grid'].values.astype(int)

    # read time
    data_time = nc_files[0].data_vars['data_time'].values
    
    
    ################ variables to be concatenated over the datasets ################
    # define empty lists of xarray.DataArray objects
    xda_list_element_na = []
    xda_list_element_coords = []
    dict_xda_list_element = {}
    dict_xda_data_wave = {}
    for nag in na_grid:
        dict_xda_list_element[nag] = []
        dict_xda_data_wave[nag] = []
    
    # loop over nc files
    for nc_file in nc_files:
        # append DataArrays
        xda_list_element_na.append(nc_file.data_vars['list_element_na'])
        xda_list_element_coords.append(nc_file.data_vars['list_element_coords'])
        for nag in na_grid:
            dict_xda_list_element[nag].append(nc_file.data_vars['list_element__NaG=%d' % nag])
            dict_xda_data_wave[nag].append(nc_file.data_vars['data_wave__NaG=%d' % nag])
            
    # concat xarray.DataArray
    xda_list_element_na = xr.concat(xda_list_element_na, dim='dim_element')
    xda_list_element_coords = xr.concat(xda_list_element_coords, dim='dim_element')
    for nag in na_grid:
        dict_xda_list_element[nag] = xr.concat(dict_xda_list_element[nag], dim='dim_element__NaG=%d' % nag)
        dict_xda_data_wave[nag] = xr.concat(dict_xda_data_wave[nag], dim='dim_element__NaG=%d' % nag)
        
    # read data to numpy.ndarray
    list_element_na = xda_list_element_na.values.astype(int)
    list_element_coords = xda_list_element_coords.values
    dict_list_element = {}
    dict_data_wave = {}
    for nag in na_grid:
        dict_list_element[nag] = dict_xda_list_element[nag].values.astype(int)
        if load_wave_data:
            dict_data_wave[nag] = dict_xda_data_wave[nag].values
        
    ############### return ################
    if load_wave_data:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_data_wave
    else:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_xda_data_wave
    

In [21]:
# sort mantle&crust element (12096, 2)
na_grid, data_time, list_element_na, list_element_coords, \
dict_list_element, dict_data_wave = read_element_output("/raid1/TomoNetData/Runs/LatinSphericalHarmonicsElasticBall0000/output/elements/full_coeff")
nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
coords_ref = list_element_coords.reshape((nelem * ngll), 2)
dist = np.sum(coords_ref**2, axis=1)
idx = np.argsort(dist)
coords_ref = coords_ref[idx]

In [22]:
coords_ref.shape

(3648, 2)

In [41]:
np.concatenate((coords_mantlecrust_ref, coords_outercore_ref, coords_innercore_ref), axis=0).shape

(16470, 2)

In [27]:
# def read_fourier_coef(Run_dir):
    Run_dir = "/raid1/TomoNetData/Runs/LatinSphericalHarmonicsElasticBall0003"
    nelement = 3648

    fourier_dim = 16

    whole_disp_coef = np.ndarray((nelement, 16, 3, 15))

    # mantle&crust
    na_grid, data_time, list_element_na, list_element_coords, \
    dict_list_element, dict_data_wave = read_element_output(f"{Run_dir}/output/elements/full_coeff")

    disp_coef = np.squeeze(dict_data_wave[fourier_dim],axis=(2,))
    nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
    coords = list_element_coords.reshape((nelem * ngll), 2)
    mapping = {tuple(point): index for index, point in enumerate(coords)}
    indices = [mapping[tuple(point)] for point in coords_ref]
    coords = coords[indices]
    assert (coords == coords_ref).all()
    whole_disp_coef[0:nelement,:,:,:] = disp_coef[indices,:,:,:]
# print('mantle&crust OK')


whole_disp_coef # nelement, nfourier, ncomponent, ntime (12096, 16, 3, 30)

array([[[[ 0.00000000e+00, -1.69460818e-28,  1.51761568e-12, ...,
           3.72004337e-12, -1.55161609e-13,  1.75627243e-12],
         [ 0.00000000e+00, -9.68730302e-32,  8.59830768e-16, ...,
          -2.06055401e-14,  2.38414079e-13, -2.21111202e-13],
         [ 0.00000000e+00,  1.51318835e-28, -1.33575399e-12, ...,
          -2.97497313e-12,  1.41615804e-13, -1.23264994e-12]],

        [[ 0.00000000e+00,  2.52413357e-30,  1.38814775e-14, ...,
          -2.01017805e-12, -1.11784432e-12, -5.20528932e-13],
         [ 0.00000000e+00, -9.21669122e-30, -1.22036975e-14, ...,
           8.81548578e-13, -4.87369097e-13, -1.60771217e-13],
         [ 0.00000000e+00, -2.15196777e-29,  7.07643105e-14, ...,
          -1.95424796e-12, -9.11838530e-13, -3.89307266e-13]],

        [[ 0.00000000e+00,  5.20042688e-30,  1.01653472e-15, ...,
          -4.75977656e-13,  1.76094748e-13, -1.15328740e-13],
         [ 0.00000000e+00, -1.67695490e-29,  6.23529684e-14, ...,
          -2.81194686e-12, -1.4378

In [31]:
# sort mantle&crust element (12096, 2)
na_grid, data_time, list_element_na, list_element_coords, \
dict_list_element, dict_data_wave = read_element_output("/raid1/TomoNetData/Runs/LatinSphericalHarmonicsPREM0000/output/elements/azimuthal_slices_mantlecrust")
nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
coords_mantlecrust_ref = list_element_coords.reshape((nelem * ngll), 2)
dist = np.sum(coords_mantlecrust_ref**2, axis=1)
idx = np.argsort(dist)
coords_mantlecrust_ref = coords_mantlecrust_ref[idx]
# sort outer core element (2052, 2)
na_grid, data_time, list_element_na, list_element_coords, \
dict_list_element, dict_data_wave = read_element_output("/raid1/TomoNetData/Runs/LatinSphericalHarmonicsPREM0000/output/elements/azimuthal_slices_outercore")
nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
coords_outercore_ref = list_element_coords.reshape((nelem * ngll), 2)
dist = np.sum(coords_outercore_ref**2, axis=1)
idx = np.argsort(dist)
coords_outercore_ref = coords_outercore_ref[idx]
# sort inner core element (2322, 2)
na_grid, data_time, list_element_na, list_element_coords, \
dict_list_element, dict_data_wave = read_element_output("/raid1/TomoNetData/Runs/LatinSphericalHarmonicsPREM0000/output/elements/azimuthal_slices_innercore")
nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
coords_innercore_ref = list_element_coords.reshape((nelem * ngll), 2)
dist = np.sum(coords_innercore_ref**2, axis=1)
idx = np.argsort(dist)
coords_innercore_ref = coords_innercore_ref[idx]

In [3]:
def read_slice_data(Run_dir):
    nelement = 3648
    nslice = 16
    whole_wave_disp = np.ndarray((nelement, nslice, 3, 15))

    # read
    na_grid, data_time, list_element_na, list_element_coords, \
    dict_list_element, dict_data_wave = read_element_output(f"{Run_dir}/output/elements/azimuthal_slices")

    wave_disp = np.squeeze(dict_data_wave[nslice],axis=(2,))
    nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
    coords = list_element_coords.reshape((nelem * ngll), 2)
    mapping = {tuple(point): index for index, point in enumerate(coords)}
    indices = [mapping[tuple(point)] for point in coords_ref]
    coords = coords[indices]
    assert (coords == coords_ref).all()
    whole_wave_disp[:,:,:,:] = wave_disp[indices,:,:,:]
    # print('mantle&crust OK')

    return data_time, whole_wave_disp # nelement, nslice, ncomponent, ntime (16470, 16, 3, 30)


In [37]:
# def save_wfslice_hdf5(StartNum,EndNum):
StartNum, EndNum= 0,1
'''
Transforming wavefield nc files and harmonics pkl files to a single HDF5 file
'''
global coords_ref
path = '../Runs'
model_name = "LatinSphericalHarmonicsElasticBall"
hdf5_file_path = f"../DataSet/wf_slice_data_{StartNum:0>4d}_{EndNum:0>4d}.h5"
num_models = EndNum-StartNum  # Total number osf models
timestep_dims = (num_models, 15)

element_coords_sz_dims = (3648, 2)
disp_dims = (num_models, 3648, 16, 3, 15) # model_id, element_id, slice_id, component_id, time_id
harmonics_dims = (num_models, 1215)
slice_dims = (16)

with h5py.File(hdf5_file_path, "w") as data_hdf5:
    # Preallocate datasets for each variable
    dtype = 'float32'
    element_coords_sz_data = data_hdf5.create_dataset("element_coords_sz", element_coords_sz_dims, dtype='float64')
    disp_data = data_hdf5.create_dataset("disp", disp_dims, dtype=dtype)
    harmonics_data = data_hdf5.create_dataset("harmonics", harmonics_dims, dtype=dtype)
    slice_data = data_hdf5.create_dataset("slice", slice_dims, dtype=dtype)
    timestep_data = data_hdf5.create_dataset("timestep", timestep_dims, dtype=dtype)

    slice_data = np.radians(np.linspace(0, 360, 17)[0:-1])

    # sort mantle&crust element (12096, 2)
    na_grid, data_time, list_element_na, list_element_coords, \
    dict_list_element, dict_data_wave = read_element_output(f"{path}/LatinSphericalHarmonicsElasticBall0000/output/elements/azimuthal_slices")
    nelem, ngll = list_element_coords.shape[0], list_element_coords.shape[1]
    coords_ref = list_element_coords.reshape((nelem * ngll), 2)
    dist = np.sum(coords_ref**2, axis=1)
    idx = np.argsort(dist)
    coords_ref = coords_ref[idx]
    element_coords_sz_data[:,:] = coords_ref[:,:]
    
    print(element_coords_sz_data.shape)

    for index, model_id in enumerate(range(StartNum, EndNum)):
        print(f"Processing model {model_id}")

        run_path = f"{path}/{model_name}{model_id:0>4d}"
        harmonics_path = f"{path}/{model_name}{model_id:0>4d}/Spherical_Harmonics.pkl"
        try:
            timestep, whole_wave_disp = read_slice_data(run_path)
            harmonics_pkl = pickle.load(open(harmonics_path, "rb"))
        except FileNotFoundError as e:
            print(f"File not found: {e.filename}")
            continue
        timestep_data[index, :] = timestep
        disp_data[index, :, :, :, :] = whole_wave_disp
        harmonics_data[index, :] = np.array(harmonics_pkl['Value'])
    print("Finished processing all models.")

(3648, 2)
Processing model 0
Finished processing all models.


In [9]:
save_wfslice_hdf5(0,1)

Processing model 0
Finished processing all models.


In [47]:
data_time

array([ 224.865 ,  524.8165,  824.768 , 1124.7195, 1424.671 , 1724.6225,
       2024.574 , 2324.5255, 2624.477 , 2924.4285, 3224.38  , 3524.3315,
       3824.283 , 4124.2345, 4424.186 , 4724.1375, 5024.089 , 5324.0405,
       5623.992 , 5923.9435])

In [38]:
import h5py

In [39]:
f = h5py.File('../DataSet/wf_slice_data_0000_0001.h5','r')

In [40]:
f.keys()

<KeysViewHDF5 ['disp', 'element_coords_sz', 'harmonics', 'slice', 'timestep']>

In [41]:
f['element_coords_sz'][:]

array([[   7.7184051 ,    6.81635405],
       [   7.7184051 ,   -6.81635405],
       [  20.44653645,    6.81017282],
       ...,
       [ 180.75188936,  970.81666884],
       [ 739.77218959, -654.13558037],
       [ 986.83085478,   36.34713265]])

In [42]:
f.close()